In [2]:
import requests # pip install requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
from lxml import etree
import re
import pandas as pd
from IPython.display import display, IFrame

In [3]:
# Specify the URL of the webpage you want to display
url = "https://vymaps.com/IN/Jharkhand/sublocality/1"

# Set the width and height of the iframe
width = "100%"
height = "400px"

# Embed the webpage within an iframe and display it
display(IFrame(url, width=width, height=height))

In [4]:
city_name = "Jharkhand"
city_pages = 1

page_num = 1
name_coordinates = []
address_list = []
tag = "br/"

names = []
coordinates = []

for each in range(city_pages):
    url = f"https://vymaps.com/IN/{city_name}/sublocality/{page_num}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    page_num+=1
    
    for link in soup.find_all('a'):
        if link.text not in ['VYMaps.com', 'Home', 'India', 'Jharkhand','About\xa0Us', 'Privacy\xa0Policy', 'Listing\xa0Submission', 'Sitemap']:
            if any(char.isdigit() for char in link.text):
                coordinates.append(link.text)
            else:
                names.append(link.text)
                
    for link in soup.find_all('p'):
        reg_str = "<" + tag + ">(.*?)<" + tag + ">"
        res = str(re.findall(reg_str, str(link)))
        address = res[11:-3]
        if address:
            address_list.append(address)
        
data = {
    'Name': names,
    'Coordinate': coordinates,
    'Address': address_list
}

df = pd.DataFrame(data)

# Print the DataFrame
df.head()

,Name,Coordinate,Address
0,Chota Panchgar,"25.2334523, 87.654205","Chota Panchgar, Sahibganj, Jharkhand 816109, I..."
1,Ganga Vihar,"25.2248175, 87.6400759","Ganga Vihar, Sahibganj PART, Jharkhand 816109,..."
2,Kalikapur,"24.6368325, 87.8623411","Kalikapur, Pakur, Jharkhand 816107, India"
3,Harindanga Bazar,"24.6352269, 87.8537478","Harindanga Bazar, Pakur, Jharkhand 816107, India"
4,Tulshi Nagar,"24.6388927, 87.8506783","Tulshi Nagar, Pakur, Jharkhand 816107, India"


In [5]:
df.to_csv(f"./vymaps_extraction/{city_name}.csv", index=False)